In [109]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

%matplotlib qt

In [110]:
# Global constants
pi = np.pi
n0 = 1; e0 = 8.85e-12; mu0 = 4*np.pi*1e-7
c = 1/np.sqrt(e0*mu0)

# Pulse definition
Nf = 200
fc = 3e9         
df = 1e9                                # Deviation frequency [Hz]
B = (fc+df) - (fc-df)                   # Bandwidth [Hz]
f = np.linspace(fc-df,fc+df,Nf)         # Frequency axis [Hz]
w, wc = 2*np.pi*f, 2*np.pi*fc            # Pulsation axis & central pulsation
sigma_w = 2*np.pi*0.2*df                # Standard deviation of pulsations

A = (1/(np.sqrt(2*np.pi)*sigma_w)) * np.exp(-(w-wc)**2/(2*(sigma_w**2)))        # Amplitude distribution over frequencies (Gaussian)

# Media parameters
l0 = 1.5            # Length of medium 0 [m]
k0 = w * n0 /c          # Wavenumber 0

n1 = 1.2                # Refractive index of medium 1
l1 = 1               # Length of medium 1 [m]
k1 = w * n1 /c          # Wavenumber 1

n2 = 1.7                # Refractive index of medium 2
l2 = 1.5              # Length of medium 2 [m]
k2 = w * n2 /c          # Wavenumber 2

n3 = n0                 # Refractive index of medium 3
l3 = 1              # Length of medium 3 [m]
k3 = w * n3 /c          # Wavenumber 3

# Axis
Nz = 100           # Precision
z0 = np.linspace(0,l0, int(l0*Nz))
z1 = np.linspace(0,l1, int(l1*Nz))
z2 = np.linspace(0,l2, int(l2*Nz))
z3 = np.linspace(0,l3, int(l3*Nz))

z = np.array([*z0, *z1 + l0, *z2 + (l0+l1), *z3 + (l0+l1+l2)])      # Global z coordinates

print(z)


[0.         0.01006711 0.02013423 0.03020134 0.04026846 0.05033557
 0.06040268 0.0704698  0.08053691 0.09060403 0.10067114 0.11073826
 0.12080537 0.13087248 0.1409396  0.15100671 0.16107383 0.17114094
 0.18120805 0.19127517 0.20134228 0.2114094  0.22147651 0.23154362
 0.24161074 0.25167785 0.26174497 0.27181208 0.28187919 0.29194631
 0.30201342 0.31208054 0.32214765 0.33221477 0.34228188 0.35234899
 0.36241611 0.37248322 0.38255034 0.39261745 0.40268456 0.41275168
 0.42281879 0.43288591 0.44295302 0.45302013 0.46308725 0.47315436
 0.48322148 0.49328859 0.5033557  0.51342282 0.52348993 0.53355705
 0.54362416 0.55369128 0.56375839 0.5738255  0.58389262 0.59395973
 0.60402685 0.61409396 0.62416107 0.63422819 0.6442953  0.65436242
 0.66442953 0.67449664 0.68456376 0.69463087 0.70469799 0.7147651
 0.72483221 0.73489933 0.74496644 0.75503356 0.76510067 0.77516779
 0.7852349  0.79530201 0.80536913 0.81543624 0.82550336 0.83557047
 0.84563758 0.8557047  0.86577181 0.87583893 0.88590604 0.89597

In [111]:
# Useful functions definition
rho     =   lambda n,np : (n-np)/(n+np)                                 # Return rho of interface n|np
tau     =   lambda n,np : (2*n)/(n+np)                                  # Return tau of interface n|np
gamma   =   lambda gammap, rho : (rho + gammap) / (1 + rho*gammap)      # Return gamma of interface rho: _ | gammap

def interface(Efp, Ebp, tau, rho):        # Propagation from E' to E
    Ef = (Efp + rho*Ebp) / tau
    Eb = (rho*Efp + Ebp) / tau
    return Ef, Eb

def propagation(Ef0, Eb0, k, z):
    Ef = Ef0 * np.exp(-1j*k*z)
    Eb = Eb0 * np.exp(1j*k*z)
    return Ef, Eb

def gamma_propagation(gamma0, k, l):
    return gamma0 * np.exp(-2*1j*k*l)       # Negative exponential because it is always used from right to left

In [112]:
# Compute all useful parameters

rho3 = rho(n2, n3)
rho3p = rho(n3, n2)
tau3 = tau(n2, n3)
tau3p = tau(n3, n2)
gamma3 = rho3

rho2 = rho(n1, n2)      # left of interface
rho2p = rho(n2, n1)     # right of interface
tau2 = tau(n1, n2)
tau2p = tau(n2, n1)

rho1 = rho(n0, n1)      # left of interface
rho1p = rho(n1, n0)     # right of interface
tau1 = tau(n0, n1)
tau1p= tau(n1, n0)


In [113]:
t = np.linspace(0,1e-6,1000)

E = np.zeros(len(z))
E_time = np.zeros((len(t),len(E)))

for i in range(len(t)):

    for j in range(len(f)):
        gamma2p = gamma_propagation(gamma3, k2[j], l2)
        gamma2 = gamma(gamma2p, rho2)

        gamma1p = gamma_propagation(gamma2, k1[j], l1)
        gamma1 = gamma(gamma1p, rho1)

        Ef0, Eb0 = propagation(1, gamma1, k0[j], z0)    # 0 and not gamma1?

        Ef1_0, Eb1_0 = interface(Ef0[-1], Eb0[-1], tau1, rho1)
        Ef1, Eb1 = propagation(Ef1_0, Eb1_0, k1[j], z1)

        Ef2_0, Eb2_0 = interface(Ef1[-1], Eb1[-1], tau2, rho2)
        Ef2, Eb2 = propagation(Ef2_0, Eb2_0, k2[j], z2)

        Ef3_0, Eb3_0 = interface(Ef2[-1], Eb2[-1], tau3, rho3)
        Ef3, Eb3 = propagation(Ef3_0, Eb3_0, k3[j], z3)

        # Reflected waves should also be taken into account
        

        E = np.array([*(Ef0+Eb0), *(Ef1+Eb1), *(Ef2+Eb2), *(Ef3+Eb3)])

        E_time[i] += A[j]*np.real(E*np.exp(1j*w[j]*t[i]))

E_time /= np.max(E_time)

In [114]:
print(E_time[0][0])

0.6266715968233497


In [117]:
c = ["tab:orange", "tab:green", "tab:red", "tab:blue"]

fig, ax = plt.subplots()

ax.set_xlabel("z [m]")
ax.set_ylabel("Electric Field [V/m]")
ax.grid()
ax.set_ylim(-1.5, 1.5)

line, = ax.plot(z, E_time[0])
yrange = plt.gca().get_ylim()


plt.fill_between(z0, yrange[0], yrange[1], color=c[1],alpha=0.3, label=r"$n=1$")
plt.fill_between(l0+z1, yrange[0], yrange[1], color=c[0],alpha=0.3, label=r"$n=$"+str(n1))
plt.fill_between(l0+l1+z2, yrange[0], yrange[1], color=c[2],alpha=0.3, label=r"$n=$"+str(n2))
plt.fill_between(l0+l1+l2+z3, yrange[0], yrange[1], color=c[1],alpha=0.3)
ax.legend(loc="upper right")

def anim(i):       
    #line.set_ydata(compute_s(z,t[i]))  # update the data
    line.set_ydata(E_time[i])
    return line,

ani = animation.FuncAnimation(fig, anim, frames=len(t),interval=500, repeat=True ,blit=False)
plt.show()

#writer = animation.PillowWriter(fps=15, metadata=dict(artist='Me'),bitrate=1800)
#ani.save('layer_paquet_wave_1f.gif', writer=writer)